In [18]:
"""
Created on Sun Apr  4 23:02:11 2021
@author: ryanp
"""

import requests
!pip install selenium
!pip install bs4
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
import pandas as pd
from collections import OrderedDict
from bs4.element import Comment
import urllib.request
import time
from selenium.webdriver.chrome.options import Options

import os

seed_urls = ['https://www.wired.com/category/business',
             'https://www.wired.com/category/backchannel',
             'https://www.wired.com/category/gear']

page = requests.get('https://www.bbc.com/')
soup = BeautifulSoup(page.content, 'html.parser')
#%% Get all sub-links on page
articles = []
hold_heads = []
headlines = []
hold_sub = []
sub_articles = []
side_articles = []

for body in soup.find_all(class_="content"): #scrape all URLs on main part of page
    for link in body.find_all("a"):
        articles.append(link.get('href'))

#for story in articles: #get side articles
 #   if ".com/story" in story:
 #       side_articles.append(story)

for story in articles: #get rest of page articles
    if "news" in story:
        hold_sub.append(story)

#for x in side_articles: #remove side articles from rest due to format
#    hold_sub.remove(x)
    
sub_articles = list(OrderedDict.fromkeys(hold_sub)) #remove duplicates by creating dictionary then convert back to list; keeps order v set

for title in soup.find_all("h3"): #headlines
    hold_heads.append(title.get_text())
headlines = list(OrderedDict.fromkeys(hold_heads))

for i in range(len(sub_articles)): #add prefix to articles for use in function
    sub_articles[i] = 'https://www.bbc.com' + sub_articles[i]


In [59]:
# instantiate a chrome options object so you can set the size and headless preference
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

chrome_driver = r'binder\chromedriver.exe'


In [60]:
def get_html_scroll(url):
    options = Options()
    options.set_headless(headless = True)
    browser = webdriver.Chrome(options = options, executable_path = chrome_driver)
    browser.get(url)
    
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(4)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            time.sleep(4)
            match=True
    post_elms = browser.page_source
    return post_elms

In [61]:
#%% Scrape sub-links content
def build_dataset (sub_articles):
    article_data = []
    for url in sub_articles:
        data = get_html_scroll(url)        
        soup = BeautifulSoup(data.content, 'html.parser')
        
        news_articles = [{'article_headline': soup.find('h1').string,
                          'article_contents': soup.find_all('p')}]
        article_data.extend(news_articles)
        #time.sleep(4)
    df = pd.DataFrame(article_data)
    df = df[['article_headline', 'article_contents']]
    return df



In [62]:
#%% run program
news_df = build_dataset(sub_articles)
news_df.head(10)

#%% trial of different method



/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use setter for headless property instead of set_headless
  This is separate from the ipykernel package so we can avoid doing imports until


WebDriverException: Message: 'binder\chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts) 
#%%

html = urllib.request.urlopen(sub_articles[1]).read()
print(text_from_html(html))